# MNIST 손글씨 - CNN v1
- 2 Conv2D layers
- 1 Fully Connected Network layer
- 1 output layer

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
seed = 2022
np.random.seed(seed)
tf.random.set_seed(seed)

- 데이터 전처리

In [2]:
from tensorflow.keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((60000, 28, 28), (60000,), (10000, 28, 28), (10000,))

In [3]:
# X data: 0~1 사이의 값,3차원 -->4차원
# X data 전처리
X_train = X_train.reshape(-1, 28,28,1) / 255.
X_test = X_test.reshape(-1,28,28) / 255.

In [4]:
# 다중분류에 해당되니깐 one- hot encoding
from tensorflow.keras.utils import to_categorical
Y_train = to_categorical(y_train)
Y_test = to_categorical(y_test)
Y_train.shape, Y_test.shape

((60000, 10), (10000, 10))

- 모델 정의, 학습, 설정, 평가

In [5]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Conv2D, Flatten
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

In [6]:
model = Sequential([
                    Conv2D(32, kernel_size =(3,3), input_shape =(28,28,1), activation= 'relu'),
                    Conv2D(64, (3,3), activation= 'relu'),
                    Flatten(), # 1차원으로 바꿀려고
                    Dense(512, activation= 'relu'),
                    Dense(10, activation= 'softmax'),
])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 24, 24, 64)        18496     
                                                                 
 flatten (Flatten)           (None, 36864)             0         
                                                                 
 dense (Dense)               (None, 512)               18874880  
                                                                 
 dense_1 (Dense)             (None, 10)                5130      
                                                                 
Total params: 18,898,826
Trainable params: 18,898,826
Non-trainable params: 0
_________________________________________________________________


In [7]:
model.compile(optimizer= 'adam', loss= 'categorical_crossentropy', metrics= ['accuracy'])

In [8]:
model_path = 'best-mnist-cnn-v1.h5'
checkpoint = ModelCheckpoint(model_path, save_best_only = True, verbose=1) # 모니터는 디폴트값
early_stop = EarlyStopping(patience=10)

In [11]:
hist = model.fit(X_train, Y_train, validation_split= 0.2,
                 epochs =100, batch_size= 200, callbacks=[checkpoint, early_stop])


Epoch 1/100
240/240 [==============================] - ETA: 0s - loss: 0.0043 - accuracy: 0.9986
Epoch 00001: val_loss did not improve from 0.04883
240/240 [==============================] - 14s 60ms/step - loss: 0.0043 - accuracy: 0.9986 - val_loss: 0.0600 - val_accuracy: 0.9879
Epoch 2/100
240/240 [==============================] - ETA: 0s - loss: 0.0040 - accuracy: 0.9988
Epoch 00002: val_loss did not improve from 0.04883
240/240 [==============================] - 14s 60ms/step - loss: 0.0040 - accuracy: 0.9988 - val_loss: 0.0549 - val_accuracy: 0.9901
Epoch 3/100
240/240 [==============================] - ETA: 0s - loss: 0.0023 - accuracy: 0.9993
Epoch 00003: val_loss did not improve from 0.04883
240/240 [==============================] - 14s 60ms/step - loss: 0.0023 - accuracy: 0.9993 - val_loss: 0.0658 - val_accuracy: 0.9880
Epoch 4/100
240/240 [==============================] - ETA: 0s - loss: 0.0038 - accuracy: 0.9987
Epoch 00004: val_loss did not improve from 0.04883
240/240 [

In [13]:
best_model = load_model(model_path)
best_model.evaluate(X_test, Y_test)

313/313 [==============================] - 2s 7ms/step - loss: 0.0403 - accuracy: 0.9861


[0.04025445878505707, 0.9861000180244446]

- MNIST 정확도
    1. 신경망 : 0.9810
    2. SVM : 0.9837
    3. CNN v1 : 0.9861